In [1]:
import pickle, gzip, json, os
import numpy as np
import matplotlib.pyplot as plt
import simtk.unit as simtk_unit
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.evaluator.client import RequestResult
from openff.evaluator.datasets import PhysicalPropertyDataSet
from openff.evaluator import unit
from matplotlib import patheffects
from matplotlib.ticker import FuncFormatter
from datetime import datetime

In [2]:
#Count the folders in the current directory.
folders = 0
tset_json = False
path = os.getcwd()
for _, dirnames, filenames in os.walk(path):
    
    for direc in dirnames:
        if direc[:5] == 'iter_':
            folders += 1
    
    #Check if a training_set.json file is present as well -- needed to graph experimental data in plot_properties
    if 'training_set.json' in filenames:
        tset_json = True

In [3]:
#Plot the value of the objective function across all iterations
def plot_objective(savefig=False, x_tick_step=1, y_tick_step=5, label_best=True, label_worst=True):
    
    #Obj is the list of values of the objective function
    obj = []
    
    #Open all objective.p files
    for i in range(folders):
        with gzip.open(f"iter_{str(i).rjust(4,'0')}/objective.p",'rb') as f:

            temp_dict = pickle.load(f)
            obj.append(temp_dict['X'])
    
    #Create plot
    lowest_x = obj.index(min(obj))
    highest_x = obj.index(max(obj))
    plt.plot(obj, 'r.-')
    
    #Cosmetic options
    plt.xticks(np.arange(0, folders, x_tick_step))
    plt.yticks(np.arange(0, max(obj)+min(obj), step=y_tick_step))
    if label_best:
        plt.text(lowest_x, obj[lowest_x], f"({lowest_x}, {round(obj[lowest_x], 5)})")
    if label_worst:
        plt.text(highest_x, obj[highest_x], f"({highest_x}, {round(obj[highest_x], 5)})")
    title = "Objective Function versus Time"
    plt.suptitle(title)
    subtitle = ""
    plt.title(subtitle)
    plt.xlabel("Iteration")
    plt.ylabel("Objective function")
    
    #If requested, save the figure
    if savefig:
        now = datetime.now()
        plt.savefig('objective_results.png', dpi=400, transparent=False, facecolor='white', metadata={'title':title, 'date':str(now)})

    plt.show()

In [12]:
#Plot the values of all properties across all iterations
def plot_properties(expt=False, savefig=False):
    
    #Check for conflict of expt and training_set
    if expt and (not tset_json):
        print("A training_set.json file is required to plot experimental data. Setting expt=False.")
        expt=False
    
    list_of_results = []
    
    for i in range(0, folders):
        #Path to this iteration's results json
        result_path = f"iter_{str(i).rjust(4, '0')}/results.json"
        with open(result_path, "r") as f:

            #Create PhysicalPropertyDataSet object from the estimated results
            result = RequestResult.from_json(result_path).estimated_properties
            list_of_results.append(result)
    
    #Declare the number of properties in each iteration
    num_properties = len(list_of_results[0])
    
    y_values = []
    x_values = list(np.arange(0, folders))
    
    for i in range(0, num_properties):
        
        y_values.append([])
        
        for j in range(0, folders):
            
            #For debugging, uncomment following line
            #print(list_of_results[j].properties[i])
            y_values[i].append(list_of_results[j].properties[i].value.m_as(unit.kcal / unit.mole))
    
        #Create a figure for this particular property
        plt.figure(i)
        
        #Plot the y_values for this property against the iterations
        plt.plot(x_values, y_values[i])
        
        #Set the title
        title = list_of_results[j].properties[i].substance.components
        plt.title(str(title))
        
        #Set the axis labels
        plt.xlabel('Iteration')
        plt.ylabel('kcal/mole')
        
        #If expt, plot the expt
        if expt:
            with open('training_set.json', 'r') as f:
                
                tset_dict = json.load(f)
                
                #To allow for unit conversion, create a pint Quantity
                expt_val_and_unit = str(tset_dict['properties'][i]['value']['value']) + str(tset_dict['properties'][i]['value']['unit'])
                expt_val_and_unit = unit.Quantity(expt_val_and_unit)
                
                #Create final, unit-adjusted expt_val
                expt_val = expt_val_and_unit.m_as(unit.kcal / unit.mole)
                
                #Plot the expt_val as a horizontal line
                plt.hlines(expt_val, xmin=min(x_values), xmax=max(x_values), linestyles='dashed', label=f'{round(expt_val_and_unit.to(unit.kcal/unit.mole), 4)}')
        
            plt.legend(loc='upper left')
        
        plt.show()
        
        #If savefig, save the graph
        if savefig:
            plt.savefig(f'graph_property_{i}.png', facecolor='white', dpi=600)
    
    #For debugging, uncomment following line
    #print(y_values)

In [5]:
def plot_parameters(savefig=False):
    
    #Place your parameters' SMIRKS here. Currently manual, needs to be automated. Only supports radius and scale right now.
    atom_type = ['[#8:1]']
    radius = {'[#8:1]':[]}
    scale = {'[#8:1]':[]}
    
    
    for i in range(0,folders):    
        force_field = ForceField(
                f"iter_{str(i).rjust(4,'0')}/openff-2.0.0-GBSA_MKG-tagged.offxml", allow_cosmetic_attributes=True
            )
        
        for atype in atom_type:
                
            radius[atype].append(force_field.get_parameter_handler("GBSA")[atype].radius.value_in_unit(simtk_unit.angstrom))
            scale[atype].append(force_field.get_parameter_handler("GBSA")[atype].scale)
    
    plt.figure(figsize=(18,12))
    
    for i, atype in enumerate(atom_type):
        
        #Uncomment to adjust the size of the overall figure
        #plt.figure(figsize=(9,6))
        
        ax = plt.subplot(3,3,i+1)
        u = np.diff(radius[atype])
        v = np.diff(scale[atype])
        pos_x = radius[atype][:-1] + u/2
        pos_y = scale[atype][:-1] + v/2
        norm = np.sqrt(u**2+v**2) 

        plt.plot(radius[atype], scale[atype], "o-",label=atype)
        plt.quiver(pos_x, pos_y, u/norm, v/norm, color='k', angles="xy", zorder=5, pivot="mid")

        def format_tick_labels(x, pos):
            return '{0:.3f}'.format(x)

        ax.yaxis.set_major_formatter(FuncFormatter(format_tick_labels))
        plt.ylabel("scale ", fontsize=14)
        plt.xlabel("radius ($\AA$)", fontsize=14)
        plt.title(f"{atype}")
        plt.tight_layout()

    #If requested, save the figure
    if savefig:
        now = datetime.now()
        title = "Parameter Results"
        plt.savefig('parameter_results.png', dpi=500, transparent=False, facecolor='white', metadata={'title':title, 'author':author, 'date':str(now)})    
        
    
    plt.show()